<a href="https://colab.research.google.com/github/KevinTheRainmaker/TF_for_Use/blob/main/01_Keras_Model_using_TFDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 케라스 모델에서 텐서플로우 공개 데이터셋 사용하기

### TFDS란?
> ***TFDS provides a collection of ready-to-use datasets for use with TensorFlow, Jax, and other Machine Learning frameworks.***
>
> \- 텐서플로우 공식 가이드

- 데이터셋을 사용하기 쉽게 제공하는 API
- 데이터를 얻는 모든 전처리 단계를 손쉽게 적용 가능

### 사용 가능한 데이터셋 종류
- **오디오:** 음성 및 음악 데이터
- **이미지:** 개-고양이 같은 간단한 데이터셋부터 당뇨망막변증 진단 등에 사용되는 고급 연구 데이터셋까지
- **객체 탐지:** COCO, OpenImages 등
- **Structured:** 타이타닉 데이터, 아마존 리뷰 등
- **문서 요약:** CNN News, Daily Mail, 과학 논문, wikiHow 등
- **텍스트:** iMDb 영화 리뷰, 자연어 질문 등
- **번역:** 다양한 번역 모델 훈련 데이터셋
- **비디오:** Moving MNIST, 스타크래프트 등

## 텐서플로우 데이터셋 시작하기

### TensorFlow Dataset

In [ ]:
!pip install -q tfds-nightly tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tfds.__version__)

4.8.0+nightly


In [ ]:
# 전체 목록
# tfds.list_builders()

### 데이터 로드하기

In [ ]:
# Fashion-MNIST 데이터셋 load
mnist_data = tfds.load('fashion_mnist')

In [ ]:
for split in mnist_data:
  print(split, type(split))

train <class 'str'>
test <class 'str'>


In [ ]:
# 실제 데이터를 담음 split
mnist_train = tfds.load(name='fashion_mnist', split='train')
assert isinstance(mnist_train, tf.data.Dataset)
print(type(mnist_train))

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


PrefetchDataset을 얻을 수 있음

In [ ]:
# Single sample
train_ds = mnist_train.take(1)

In [ ]:
for item in train_ds:
  print(type(item))
  print(item.keys())

<class 'dict'>
dict_keys(['image', 'label'])


각 레코드의 타입은 dictionary이며, key 값은 image와 label임을 알 수 있다

In [ ]:
for item in train_ds:
  print(type(item['image']))
  print(type(item['label']))  
  print(tf.shape(item['image']))
  print(item['label'])

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([28 28  1], shape=(3,), dtype=int32)
tf.Tensor(2, shape=(), dtype=int64)


이미지는 28 x 28 사이즈의 tf.Tensor 타입 배열

레이블은 클래스 번호

### 좀더 자세한 데이터셋 정보 얻기

In [ ]:
mnist_test, info = tfds.load(name='fashion_mnist', with_info='true')

In [ ]:
print(info)

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='/root/tensorflow_datasets/fashion_mnist/3.0.1',
    file_format=tfrecord,
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    

## 케라스 분류 모델에서 텐서플로 데이터셋 사용하기

In [ ]:
(train_data, test_data), info = tfds.load('horses_or_humans', split=['train','test'], as_supervised=True, with_info=True)

In [ ]:
print(info)

tfds.core.DatasetInfo(
    name='horses_or_humans',
    full_name='horses_or_humans/3.0.0',
    description="""
    A large set of images of horses and humans.
    """,
    homepage='http://laurencemoroney.com/horses-or-humans-dataset',
    data_path='/root/tensorflow_datasets/horses_or_humans/3.0.0',
    file_format=tfrecord,
    download_size=153.59 MiB,
    dataset_size=153.53 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=256, num_shards=1>,
        'train': <SplitInfo num_examples=1027, num_shards=2>,
    },
    citation="""@ONLINE {horses_or_humans,
    author = "Laurence Moroney",
    title = "Horses or Humans Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/horses-or-humans-dataset"
    }""",
)


(300, 300, 3) 크기의 train image 1,027장과 test image 256장

In [ ]:
train_batches = train_data.shuffle(100).batch(100)
test_batches = test_data.batch(100)

In [ ]:
# Model Build
import tf,keras.layers as layers
model = tf.keras.models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

])